#### Pytorch Neural Network Module  
The torch.nn module is a core library that provides a wide array of classes and funcions designed to build neural networks efficiently.  
Components:  
1. Module(layers)  
2. Activation Funtions  
3. Loss functions  
4. Container Modules (eg. nn.Sequential)  
5. Regularization and Dropout  


In [14]:
import torch
import torch.nn as nn
from torchinfo import summary
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

In [7]:
#creating a simple neural network
class Model(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.linear = nn.Linear(num_features, 1)
        self.sigmoid =nn.Sigmoid()
    
    def forward(self, features):
        out = self.linear(features)
        out = self.sigmoid(out)
        return out
    

In [8]:
#sample dat creation
features = torch.rand(10,5)

#model object creation
model = Model(features.shape[1])

#call the model for forward pass
model(features)


tensor([[0.4004],
        [0.3681],
        [0.4119],
        [0.4021],
        [0.4149],
        [0.3471],
        [0.3542],
        [0.3792],
        [0.4590],
        [0.4347]], grad_fn=<SigmoidBackward0>)

In [9]:
#checking values of weights and bias
model.linear.weight, model.linear.bias

(Parameter containing:
 tensor([[ 0.1066, -0.2962, -0.1548,  0.1548, -0.3197]], requires_grad=True),
 Parameter containing:
 tensor([-0.1128], requires_grad=True))

In [11]:
#model information using torchinfo.summary
summary(model, input_size = features.shape)

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [10, 1]                   --
├─Linear: 1-1                            [10, 1]                   6
├─Sigmoid: 1-2                           [10, 1]                   --
Total params: 6
Trainable params: 6
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

##### A more complex neural network using nn module using nn.sequential

In [12]:
class Model(nn.Module):
    def __init__(self,num_features):
        super().__init__()
        self.network = nn.Sequential(
        nn.Linear(num_features, 5),
        nn.ReLU(),
        nn.Linear(5, 1),
        nn.Sigmoid()
        )
    
    def forward(self,features):
        out = self.network(features)
        return out


In [13]:
#sample dat creation
features = torch.rand(10,5)

#model object creation
model = Model(features.shape[1])

#call the model for forward pass
model(features)


tensor([[0.5453],
        [0.5546],
        [0.5549],
        [0.5641],
        [0.5504],
        [0.5547],
        [0.5120],
        [0.5458],
        [0.5680],
        [0.5425]], grad_fn=<SigmoidBackward0>)

#### Training a complex neural network using cancer dataset

In [16]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df = df.drop(["id","Unnamed: 32"],axis=1)
df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0})
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [36]:
#Splitting the dataframe into train and test sets
X = df.drop("diagnosis", axis=1)
y = df["diagnosis"]

ss= StandardScaler()
X = ss.fit_transform(X.values)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

x_train_tensor = torch.from_numpy(x_train).float()
y_train_tensor = torch.from_numpy(y_train.values).float()
x_test_tensor = torch.from_numpy(x_test).float()
y_test_tensor = torch.from_numpy(y_test.values).float()

In [37]:
class myModel(nn.Module):
    def __init__(self,x):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(x, 5),
            nn.ReLU(),
            nn.Linear(5, 1),
            nn.Sigmoid()
        )
    
    def forward(self,x):
        out = self.network(x)
        return out
     

In [39]:
model = myModel(x_train_tensor.shape[1])

epochs = 25
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(epochs):
    
    #forward_pass
    y_pred = model(x_train_tensor)
    
    #loss_calculation
    loss = loss_fn(y_pred, y_train_tensor.reshape(-1, 1))

    #zero_grad
    optimizer.zero_grad()
    
    #backward_pass
    loss.backward()
    
    #parameter_update_using_optimizer
    with torch.no_grad():
        optimizer.step()
    

    print(f"Loss after epoch {epoch+1}: {loss.item()}")



Loss after epoch 1: 0.6512182354927063
Loss after epoch 2: 0.646933913230896
Loss after epoch 3: 0.642687201499939
Loss after epoch 4: 0.6384835243225098
Loss after epoch 5: 0.6343203783035278
Loss after epoch 6: 0.6302047371864319
Loss after epoch 7: 0.6261313557624817
Loss after epoch 8: 0.622100830078125
Loss after epoch 9: 0.6181157827377319
Loss after epoch 10: 0.6141701340675354
Loss after epoch 11: 0.6102798581123352
Loss after epoch 12: 0.6064416170120239
Loss after epoch 13: 0.6026518940925598
Loss after epoch 14: 0.5989047288894653
Loss after epoch 15: 0.5951956510543823
Loss after epoch 16: 0.5915102362632751
Loss after epoch 17: 0.587852954864502
Loss after epoch 18: 0.5842248797416687
Loss after epoch 19: 0.5806311964988708
Loss after epoch 20: 0.5770655274391174
Loss after epoch 21: 0.5735034942626953
Loss after epoch 22: 0.5699598789215088
Loss after epoch 23: 0.5664276480674744
Loss after epoch 24: 0.5629016160964966
Loss after epoch 25: 0.5593814849853516


In [40]:
with torch.no_grad():
    y_pred_test = model(x_test_tensor)
    y_pred_test = (y_pred_test > 0.5).float()


cm = confusion_matrix(y_test_tensor, y_pred_test)
accuracy = accuracy_score(y_test_tensor, y_pred_test)

print("Confusion Matrix:")
print(cm)
print("Accuracy:", accuracy)    

Confusion Matrix:
[[49 22]
 [ 4 39]]
Accuracy: 0.7719298245614035
